# Generate simulated data to link

In this case study, we imagine running PVS on the 2030 Census Unedited File (CUF) -- see the main notebook for more details, including references used throughout this notebook.
This notebook creates input (CUF) and reference files approximating what would be used in such a PVS process.

In [1]:
# DO NOT EDIT if this notebook is not called generate_simulated_data_small_sample.ipynb!
# This notebook is designed to be run with papermill; this cell is tagged 'parameters'
# If running with the default parameters, you can overwrite this notebook; otherwise,
# save it to another filename.
# TODO: Rename the notebook to omit 'small_sample' in the filename and omit all outputs
# from the 'canonical version'
data_to_use = 'small_sample'
output_dir = 'output'
compute_engine = 'pandas'
num_jobs = 3
memory_per_job = "50GB"

In [2]:
! date

Wed Sep 27 11:29:41 AM PDT 2023


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
import pseudopeople as psp
import numpy as np
import os
import logging

In [5]:
psp.__version__

'0.7.1'

In [6]:
# Importing pandas for access, regardless of whether we are using it as the compute engine
import pandas

In [7]:
if compute_engine == 'pandas':
    import pandas as pd
elif compute_engine.startswith('modin'):
    if compute_engine.startswith('modin_dask_'):
        import modin.config as modin_cfg
        modin_cfg.Engine.put("dask") # Use dask instead of ray (which is the default)

        import dask

        if compute_engine == 'modin_dask_distributed':
            from dask_jobqueue import SLURMCluster

            num_processes_per_job = 1
            cluster = SLURMCluster(
                queue='long.q',
                account="proj_simscience",
                # If you give dask workers more than one core, they will use it to
                # run more tasks at once.
                # There doesn't appear to be an easy way to have more than one thread
                # per worker but use them all for multi-threading code in one task
                # at a time (note that processes=1 does *not* do this).
                # It can be done with custom resource definitions, but that seems like
                # much more trouble than it's worth, since it would require modifying
                # both pseudopeople and Modin to use them.
                cores=1,
                memory=memory_per_job,
                walltime="10-00:00:00",
                # Dask distributed looks at OS-reported memory to decide whether a worker is running out.
                # If the memory allocator is not returning the memory to the OS promptly (even when holding onto it
                # is smart), it will lead Dask to make bad decisions.
                # By default, pyarrow uses jemalloc, but I could not get that to release memory quickly.
                # Even this doesn't seem to be completely working, but in combination with small-ish partitions
                # it seems to do okay -- unmanaged memory does seem to shrink from time to time, which it wasn't
                # previously doing.
                job_script_prologue="export ARROW_DEFAULT_MEMORY_POOL=system\nexport MALLOC_TRIM_THRESHOLD_=0",
            )

            cluster.scale(n=num_jobs)
            # Supposedly, this will start new jobs if the existing
            # ones fail for some reason.
            # https://stackoverflow.com/a/61295019
            cluster.adapt(minimum_jobs=num_jobs, maximum_jobs=num_jobs)

            from distributed import Client
            client = Client(cluster)
        else:
            from distributed import Client
            cpus_available = int(os.environ['SLURM_CPUS_ON_NODE'])
            client = Client(n_workers=int(cpus_available / 2), threads_per_worker=2)

        # Why is this necessary?!
        # For some reason, if I don't set NPartitions, it seems to default to 0?!
        num_row_groups = 334
        modin_cfg.NPartitions.put(min(num_jobs * num_processes_per_job * 3, num_row_groups))

        display(client)
    elif compute_engine == 'modin_ray':
        # Haven't worked on distributing this across multiple nodes
        import ray
        ray.init(runtime_env={'env_vars': {'__MODIN_AUTOIMPORT_PANDAS__': '1'}}, num_cpus=int(os.environ['SLURM_CPUS_ON_NODE']))
    else:
        # Use serial Python backend (good for debugging errors)
        import modin.config as modin_cfg
        modin_cfg.IsDebug.put(True)

    import modin.pandas as pd

    # https://modin.readthedocs.io/en/stable/usage_guide/advanced_usage/progress_bar.html
    from modin.config import ProgressBar
    ProgressBar.enable()

## Load simulated data

In [8]:
assert data_to_use in ('small_sample', 'usa')
pseudopeople_input_dir = None if data_to_use == 'small_sample' else '/mnt/team/simulation_science/priv/engineering/vivarium_census_prl_synth_pop/results/release_02_yellow/full_data/united_states_of_america/2023_07_28_08_33_09/final_results/2023_08_16_09_58_54/pseudopeople_input_data_usa/'

### Noise configuration

In order to give ourselves more of a challenge, we significantly increase the amount of noise
from the pseudopeople defaults.

In [9]:
default_configuration = psp.get_config()

In [10]:
# Helper functions for changing the default configuration according to a pattern
def column_noise_value(dataset, column, noise_type, default_value):
    if dataset in ('decennial_census', 'taxes_w2_and_1099', 'social_security'):
        if noise_type == "make_typos":
            if column == "middle_initial":
                # 5% of middle initials (which are all a single token anyway) are wrong.
                return {"cell_probability": 0.05, "token_probability": 1}
            elif column in ("first_name", "last_name", "street_name"):
                # 10% of these text columns were entered carelessly, at a rate of 1 error
                # per 10 characters.
                # The pseudopeople default is 1% careless.
                return {"cell_probability": 0.1, "token_probability": 0.1}
        elif noise_type == "write_wrong_digits":
            # 10% of number columns were written carelessly, at a rate of 1 error
            # per 10 characters.
            # The pseudopeople default is 1% careless.
            # Note that this is applied on top of (the default lower levels of) typos,
            # since typos also apply to numeric characters.
            return {"cell_probability": 0.1, "token_probability": 0.1}

    return default_value


def row_noise_value(dataset, noise_type, default_value):
    return default_value

In [11]:
custom_configuration = {
    dataset: {
        noise_category: (
            ({
                column: {
                    noise_type: column_noise_value(dataset, column, noise_type, noise_type_config)
                    for noise_type, noise_type_config in column_config.items()
                }
                for column, column_config in noise_category_config.items()
            }
            if noise_category == "column_noise" else
            {
                noise_type: row_noise_value(dataset, noise_type, noise_type_config)
                for noise_type, noise_type_config in noise_category_config.items()
            })
        )
        for noise_category, noise_category_config in dataset_config.items()
    }
    for dataset, dataset_config in default_configuration.items()
}

### Record ID tracking (data lineage)

We do a little bit of work here to enable tracking the "ground truth" (the simulant IDs from
pseudopeople).
We give each pseudopeople record/row a unique identifier for tracking, and then we immediately
separate the ground truth information (which we would not have if we were using real data)
from the rest of the columns (which we would have).
The ground truth is only used in the specific "ground truth" section of this notebook,
to help avoid accidentally leaking information into the case study.

Since we also combine/aggregate pseudopeople records as part of the process of generating the
simulated PVS reference files, ground truth is a bit more complicated than you might imagine.
For example, the ground truth may tell us that a single row in a reference file is actually
a composite of several individuals, because even the deterministic linkage (by SSN) we use
here is not without error.

We handle this by tracking *all* source records used in the construction of each record in
our reference files.
This is achieved by having a table mapping composite record IDs to the "source record IDs"
(IDs of records that were directly generated by pseudopeople).
When we aggregate records, this is combined accordingly.

In [12]:
def add_unique_record_id(df, dataset_name):
    df = df.reset_index().rename(columns={'index': 'record_id'})
    df['record_id'] = f'{dataset_name}_' + df.record_id.astype(str)
    return df


# Initializes a table listing the pairs between record_ids and source record_ids.
# Should only be called on "source records"; that is, records that
# come directly out of pseudopeople.
def record_id_to_single_source_record_pairs(df, source_col='record_id'):
    if source_col == 'record_id':
        # We can't have duplicate column names, so we make a new column
        # literally called 'source_col'
        df = df.assign(source_col=lambda df: df[source_col])
        source_col = 'source_col'

    return df[['record_id', source_col]].rename(columns={source_col: 'source_record_id'})

In [13]:
# Operations that aggregate records, combining the source_record_ids column
# between all records that are aggregated into a single row

def merge_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix, *args, **kwargs):
    assert len(dfs) == len(source_record_pairings)
    for df in dfs:
        assert 'record_id' in df.columns

    result = dfs[0]
    source_record_pairs = source_record_pairings[0]
    dfs_and_source_record_pairs_to_combine = list(zip(dfs[1:], source_record_pairings[1:]))
    for index, (df_to_merge, source_record_pairs_to_merge) in enumerate(dfs_and_source_record_pairs_to_combine):
        result = (
            result.merge(df_to_merge, *args, **kwargs)
        )
        if index == len(dfs_and_source_record_pairs_to_combine) - 1:
            # Since this is the last step, these are the record_ids that will actually be returned
            accumulate_step_record_id_prefix = new_record_id_prefix
        else:
            # A dummy intermediate -- this shouldn't be exposed to the user
            accumulate_step_record_id_prefix = f'merge_iter_{index}'

        result = add_unique_record_id(result, accumulate_step_record_id_prefix)
        source_record_pairs = pd.concat([
            # The pairs that were already in result
            source_record_pairs
                .rename(columns={'record_id': 'record_id_x'})
                .merge(result[['record_id', 'record_id_x']], on='record_id_x')
                .drop(columns=['record_id_x']),
            # The new ones
            source_record_pairs_to_merge
                .rename(columns={'record_id': 'record_id_y'})
                .merge(result[['record_id', 'record_id_y']], on='record_id_y')
                .drop(columns=['record_id_y']),
        ])
        result = result.drop(columns=['record_id_x', 'record_id_y'])

    return result, source_record_pairs


def dedupe_preserving_source_records(df, source_record_pairs, columns_to_dedupe, new_record_id_prefix):#, source_records_col='source_record_ids'):
    result = df[columns_to_dedupe].drop_duplicates()
    result = add_unique_record_id(result, new_record_id_prefix)
    df_to_result_mapping = (
        df[['record_id'] + columns_to_dedupe]
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(result, on=columns_to_dedupe)
            [['record_id', 'record_id_pre_dedupe']]
    )
    result_source_record_pairs = (
        source_record_pairs
            .rename(columns={'record_id': 'record_id_pre_dedupe'})
            .merge(df_to_result_mapping, on='record_id_pre_dedupe')
            .drop(columns=['record_id_pre_dedupe'])
    )
    return result, result_source_record_pairs


def concat_preserving_source_records(dfs, source_record_pairings, new_record_id_prefix):
    dfs = [df.rename(columns={'record_id': 'record_id_pre_concat'}) for df in dfs]
    result = pd.concat(dfs, ignore_index=True)
    result = add_unique_record_id(result, new_record_id_prefix)

    record_id_mapping = (
        result[['record_id', 'record_id_pre_concat']]
    )
    result_source_record_pairs = (
        pd.concat(source_record_pairings, ignore_index=False)
            .rename(columns={'record_id': 'record_id_pre_concat'})
            .merge(record_id_mapping, on='record_id_pre_concat', validate='m:1')
            .drop(columns=['record_id_pre_concat'])
    )

    return result.drop(columns=['record_id_pre_concat']), result_source_record_pairs

### SSA Numident

Wagner and Layne, p.4:

> The reference files are derived from the Social Security Administration
    (SSA) Numerical Identification file (SSA Numident). The Numident contains all
    transactions recorded against one Social Security Number (SSN)...

Based on the [SSA Numident through 2007 which is publicly available from NARA](https://aad.archives.gov/aad/series-description.jsp?s=5057),
we know there are three kinds of transactions: SSN applications, deaths, and claiming benefits.
SSN holders may change their information (e.g. changing name or sex) by submitting another application,
which generates an additional application transaction.
(The policies about this are found [on the SSA website](https://secure.ssa.gov/poms.nsf/lnx/0110212200).)

The paper ["Likely Transgender Individuals in U.S. Federal Administrative Records and the 2010 Census" by Benjamin Cerf Harris](https://www.census.gov/content/dam/Census/library/working-papers/2015/adrm/carra-wp-2015-03.pdf)
includes some helpful statistics (Table 2).
The average person in the SSA Numident has 2.2 transactions (called "claims" in that paper, but with the same definition
as our term "transaction": "Any time an SSN is created or information associated with an existing SSN is changed, that event is registered
as a claim.").

pseudopeople does not currently include correction, name change, or benefits claim transactions.
It only includes SSN creation and death of the SSN holder.

I've figured that there would be some delay in getting the Numident -- so by Census processing time
for the 2030 Census, only the SSA transactions by the end of 2029 would be available.
Note that with pseudopeople's current design it is only possible to set this cutoff at the end of a calendar year.
The NORC report says that "the Census NUMIDENT is recreated each year, to reflect
Social Security transaction records through **March** of each year" (p. 105),
though it isn't clear when in the year the Census Numident is actually re-created.

The SSA Numident is supposed to contain a sex column, but it currently doesn't in pseudopeople.

In [14]:
psp_kwargs = {
    'config': custom_configuration,
    'source': pseudopeople_input_dir,
}
if 'modin' in compute_engine:
    engine_kwargs['engine'] = 'modin'

In [15]:
%%time

ssa_numident = psp.generate_social_security(
    year=2029,
    **psp_kwargs,
)
ssa_numident = add_unique_record_id(ssa_numident, 'ssa_numident')
ssa_numident_source_record_pairs = record_id_to_single_source_record_pairs(ssa_numident)
ssa_numident

CPU times: user 2.92 s, sys: 77.7 ms, total: 3 s
Wall time: 3.04 s


,record_id,simulant_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date
0,ssa_numident_0,0_19979,786-77-6454,Evelyn,Nancy,Hinww,19191204,Female,creation,19191204
1,ssa_numident_1,0_6846,688-88-6377,George,Robert,Dickens,19210616,Male,creation,19211606
2,ssa_numident_2,0_19983,651-33-9561,Beatrise,Jennie,Fackler,19220113,Female,creation,19220113
3,ssa_numident_3,0_262,665-25-7858,Eura,Nadine,Crusen,19220305,Female,creation,NaN
4,ssa_numident_4,0_12473,875-10-2359,Roberta,Ruth,Muilenburg,19220306,Female,creation,19220306
...,...,...,...,...,...,...,...,...,...,...
19866,ssa_numident_19866,0_23602,203-10-6855,Birdie,Lizzy,Mercer,20291223,Female,creation,20291223
19867,ssa_numident_19867,0_23588,505-88-8814,Levi,Holden,Small,20291224,Male,creation,20291224
19868,ssa_numident_19868,0_23589,417-18-3811,Alan,Messiah,Bazan,20291227,Male,creation,20291227
19869,ssa_numident_19869,0_23596,450-74-3175,Emmi,Everlynn,Will,20291228,Female,creation,20291228


In [16]:
ssa_numident_ground_truth = ssa_numident.set_index('record_id').simulant_id
ssa_numident = ssa_numident.drop(columns=['simulant_id'])

### 1040 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files -- see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [17]:
tax_years = list(range(2025, 2030))
tax_years

[2025, 2026, 2027, 2028, 2029]

In [18]:
%%time

# Combine 1040 for all years, adding a tax_year column to track which tax year each row came from.
taxes_1040 = pd.concat([
    psp.generate_taxes_1040(
        year=year,
        **psp_kwargs,
    ).assign(tax_year=year, tax_form='1040') for year in tax_years
], ignore_index=True)
taxes_1040 = add_unique_record_id(taxes_1040, '1040')
taxes_1040_source_record_pairs = record_id_to_single_source_record_pairs(taxes_1040)
taxes_1040

CPU times: user 33.1 s, sys: 871 ms, total: 34 s
Wall time: 32.5 s


,record_id,simulant_id,household_id,first_name,middle_initial,last_name,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,...,dependent_2_last_name,dependent_2_ssn,dependent_3_first_name,dependent_3_last_name,dependent_3_ssn,dependent_4_first_name,dependent_4_last_name,dependent_4_ssn,tax_year,tax_form
0,1040_0,0_5623,0_2289,Heather,J,Campusano Robledo,794-23-1522,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,1040
1,1040_1,0_11911,0_10411,Gentle,J,Nicholas,004-53-9627,1934,cypress ave,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,1040
2,1040_2,0_25,0_16,Heather,S,Blackwell,273-52-7750,57,shadowood dr,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,1040
3,1040_3,0_15029,0_6055,Candy,S,Luedeman,384-45-8523,10881,maple av,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025,1040
4,1040_4,0_18706,0_7520,Patrick,J,Morfin,858-84-4517,103,71st st w,NaN,...,Morfin,691-64-4425,Nathan,Morfin,539-57-3727,NaN,NaN,NaN,2025,1040
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39114,1040_39114,0_21814,0_4732,Akari,L,Parkinson,595-04-6651,2921,curtis la,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2029,1040
39115,1040_39115,0_22705,0_11188,Jada,T,Franco Resendez,523-63-9324,1920,white ibis ct,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2029,1040
39116,1040_39116,0_16072,0_6479,Bianca,L,Sargent,187-01-0690,12613,teppert,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2029,1040
39117,1040_39117,0_23238,0_11927,Ashton,N,Cardenas,808-04-3639,116,wakema rd,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2029,1040


In [19]:
taxes_1040_ground_truth = taxes_1040.set_index('record_id').simulant_id
taxes_1040 = taxes_1040.drop(columns=['simulant_id'])

### W2/1099 tax filings

We assume that the last 5 years of taxes would be available and used in the construction of the reference files --
see section about reference files below.

Note that these are retrieved by *tax* year, so the 2029 taxes would be available in early 2030
(around when our hypothetical case study is taking place).

In [20]:
%%time

# Combine W2/1099 for all years, adding a tax_year column to track which tax year each row came from.
w2_1099 = pd.concat([
    psp.generate_taxes_w2_and_1099(
        year=year,
        **psp_kwargs,
    ).assign(tax_year=year) for year in tax_years
], ignore_index=True)
w2_1099 = add_unique_record_id(w2_1099, 'w2_1099')
w2_1099_source_record_pairs = record_id_to_single_source_record_pairs(w2_1099)
w2_1099

CPU times: user 27.3 s, sys: 700 ms, total: 28 s
Wall time: 27.1 s


,record_id,simulant_id,household_id,employer_id,ssn,wages,employer_name,employer_street_number,employer_street_name,employer_unit_number,...,last_name,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode,tax_form,tax_year
0,w2_1099_0,0_5,0_8,12,854-13-6295,27185,Jj Rubys Salon Studios,1300,windsor lane,NaN,...,Stark Lozano,1501,interlake ave n,NaN,NaN,Anytown,WA,00000,W2,2025
1,w2_1099_1,0_5,0_8,60,854-13-6295,7920,Freeway Insurance Agency,1105,largess ln,NaN,...,Stark Lozano,1501,interlake ave n,NaN,NaN,Anytown,WA,00000,W2,2025
2,w2_1099_2,0_5623,0_2289,11,794-23-1522,95715,Prenatal,10949,w diamond bell ranch rd,NaN,...,Campusano Robledo,NaN,NaN,NaN,14011,Anytown,WA,00000,W2,2025
3,w2_1099_3,0_7252,0_2957,69,093-85-3512,1616,Rancho Vistoso Trails Mental Health,2101,w 4th pl,NaN,...,Velazquez,927,broomfield ln,NaN,NaN,Anytown,WA,00000,W2,2025
4,w2_1099_4,0_7252,0_2957,95,093-85-3512,2683,Pikes Creek Campground,6300,east lincoln ave,NaN,...,Velazquez,927,broomfield ln,NaN,NaN,Anytown,WA,00000,W2,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53619,w2_1099_53619,0_8910,0_3621,53,119-02-0252,121064,A Car Title Loans,6100,e ball rd,NaN,...,Hurt,3515,javello rd,NaN,NaN,Anytown,WA,00000,NaN,2029
53620,w2_1099_53620,0_2378,0_12207,27,079-75-0734,34645,The Wicked Pho Licious Burgers,108,normandy lane,NaN,...,Washington,9358,e 58th st,NaN,NaN,Anytown,WA,00000,W2,2029
53621,w2_1099_53621,0_2378,0_12207,54,079-75-0734,NaN,BP,4523,lawrence rd,NaN,...,Washington,9358,e 58th st,NaN,NaN,Anytown,WA,00000,W2,2029
53622,w2_1099_53622,0_22909,0_11487,30,z87-19-2431,41695,Warrensburg,1830,kittery pt,NaN,...,Ehle,NaN,rdg park dr,NaN,NaN,Anytown,WA,00000,W2,2029


In [21]:
w2_1099_ground_truth = w2_1099.set_index('record_id').simulant_id
w2_1099 = w2_1099.drop(columns=['simulant_id'])

In [22]:
taxes, taxes_source_record_pairs = concat_preserving_source_records(
    [taxes_1040, w2_1099],
    [taxes_1040_source_record_pairs, w2_1099_source_record_pairs],
    new_record_id_prefix='taxes',
)

In [23]:
taxes_ground_truth = pd.concat([taxes_1040_ground_truth, w2_1099_ground_truth])

In [24]:
# "... many of the [IRS] records contain only the first four letters of the last name."
# (Brown et al. 2023, p.30, footnote 19)
# This should be updated in pseudopeople but for now we do it here.
# Note that this truncation only matters for ITIN PIKing since for SSNs that are present in SSA we use name from SSA.
PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION = 0.4 # is this a good guess at "many" in the quote above?
idx_to_truncate = taxes.sample(frac=PROPORTION_OF_IRS_RECORDS_WITH_TRUNCATION, random_state=1234).index
taxes.loc[idx_to_truncate, 'last_name'] = taxes.loc[idx_to_truncate, 'last_name'].str[:4]
taxes.loc[idx_to_truncate, 'last_name']

76856    Lang
62755    Wile
45       Magw
80316    Smit
73427    Hami
         ... 
67877    Hebe
47283    Hans
81811    Thru
44934    Ring
44293    Kenn
Name: last_name, Length: 37097, dtype: object

### 2030 Census Unedited File (CUF)

For now, we gloss over the data schema for addresses.
We don't know how addresses would be formatted in the CUF (and it's hard to guess, because
address is not part of the Census form), but it likely would have some of these fields
(street number, street name, etc) combined.

While PVS input files do not in general have names split into first, middle, and last,
I am guessing the CUF **would** have first name, middle initial, last name (which is how pseudopeople
generates it), because that [matches the Census questionnaire](https://www2.census.gov/programs-surveys/decennial/2020/technical-documentation/questionnaires-and-instructions/questionnaires/2020-informational-questionnaire-english_DI-Q1.pdf).

In [25]:
%%time

census_2030 = psp.generate_decennial_census(
    year=2030,
    **psp_kwargs,
)
census_2030 = add_unique_record_id(census_2030, 'census_2030')
census_2030_source_record_pairs = record_id_to_single_source_record_pairs(census_2030)
census_2030

CPU times: user 4.4 s, sys: 139 ms, total: 4.54 s
Wall time: 4.44 s


,record_id,simulant_id,household_id,first_name,middle_initial,last_name,age,date_of_birth,street_number,street_name,unit_number,city,state,zipcode,housing_type,relationship_to_reference_person,sex,race_ethnicity,year
0,census_2030_0,0_923,0_8033,Gerald,R,Allen,86,11/03/1943,1130,mallory ln,NaN,Anytown,WA,00000,Household,Reference person,Male,Black,2030
1,census_2030_1,0_2348,0_1066,April,S,Hayden,33,10/23/1996,32597,delacorte dr,NaN,Anytown,WA,00000,Household,Other nonrelative,Female,Black,2030
2,census_2030_2,0_2641,0_1066,Loretta,T,Lowe,71,06/01/1958,32597,delacorte dr,NaN,Anytown,WA,00000,Household,Reference person,Female,White,2030
3,census_2030_3,0_6176,0_2514,Sandra,S,Sorrentino,75,03/18/1954,4458,windsor pl,NaN,Anytown,WA,00000,Household,NaN,Female,Multiracial or Other,2030
4,census_2030_4,0_10251,0_5627,Bobby,S,Baker,54,05/20/1985,17868,winding trail rd,NaN,Anytown,WA,00000,Household,Other nonrelative,Male,White,2030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11018,census_2030_11018,0_17003,0_10778,Jeremy,Y,Boyd,46,07/01/1983,211,quiet way,NaN,Anytown,WA,00000,Household,Reference person,Male,Black,2030
11019,census_2030_11019,0_19380,0_11001,Wendy,M,Gross,54,12/05/1975,2801,blue rdg dr n,NaN,Anytown,WA,00000,Household,Reference person,Female,White,2030
11020,census_2030_11020,0_20272,0_5308,Ember,H,Samuels,10,10/26/2019,24113,lauder,NaN,Anytown,WA,00000,Household,Reference person,Female,Black,2030
11021,census_2030_11021,0_21997,0_10693,Athena,V,Deshpande,27,07/05/2002,1534,bentley dr,NaN,Anytown,WA,00000,Household,Reference person,NaN,Asian,2030


In [26]:
census_2030_ground_truth = census_2030.set_index('record_id').simulant_id
census_2030 = census_2030.drop(columns=['simulant_id'])

In [27]:
source_record_ground_truth = pd.concat([
    ssa_numident_ground_truth,
    taxes_ground_truth,
    census_2030_ground_truth,
])
source_record_ground_truth

record_id
ssa_numident_0       0_19979
ssa_numident_1        0_6846
ssa_numident_2       0_19983
ssa_numident_3         0_262
ssa_numident_4       0_12473
                      ...   
census_2030_11018    0_17003
census_2030_11019    0_19380
census_2030_11020    0_20272
census_2030_11021    0_21997
census_2030_11022    0_23386
Name: simulant_id, Length: 123637, dtype: object

## Create reference files

> The Census Numident – all Social Security Administration (SSA) Numident SSN records are
  edited (collapsed) to produce a Census Numident file that contains “one best-data record” for
  each SSN. All variants of name information for each SSN are retained in the Alternate Name
  Numident file, while all variants of date of birth data are retained in the Alternate DOB
  Numident. The SSN-PIK crosswalk file is used to attach a corresponding unique PIK value for
  each SSN value in the Census Numident file.

### Census Numident

Luque and Wagner, p. 4:
  
> The SSA Numident file contains all transactions ever recorded against any single SSN - with each entry
representing an addition or change (such as name changes) to the SSN record. This file is edited to
create the **Census Numident**, which contains one record for each SSN. Each SSN record in the Census
Numident contains name, DOB, sex, race, place of birth, parents’ name, citizenship status and date of death information.

and in footnote 5:

> Name edits, DOB reconciliation, and race identifiers are some of the edits conducted to produce this Numident
file. **The resulting Numident file contains the most recent name and DOB data.**

We are missing quite a few columns, since they are missing in pseudopeople's SSA Numident: sex, race, place of birth, parents' name,
citizenship status.
However, I'm pretty sure that out of these columns, only sex is used in the PVS linking process.

In [28]:
def fill_dates(df, fill_with):
    return (
        # Replace invalid dates with nans
        pd.to_datetime(df.event_date, format='%Y%m%d', errors='coerce')
            .fillna(pd.to_datetime('2100-01-01' if fill_with == 'latest' else '1900-01-01'))
    )

def best_data_from_columns(df, columns, best_is_latest=True):
    # We don't want to throw out events with a missing/invalid date, so we'll fill them with the value *least* likely to be chosen
    # (earlier than all values if taking the latest, later than all values if taking the earliest).
    fill_with = 'earliest' if best_is_latest else 'latest'

    result = (
        df
            # Without mutating the existing date column, get one that is actually
            # a date type and can be used for sorting.
            # Note: we actually convert this to an integer for sorting purposes, because Modin was having trouble
            # sorting by it as an actual datetime
            .assign(event_date_for_sort=lambda df: fill_dates(df, fill_with=fill_with).astype(np.int64) // 10 ** 9)
            .sort_values('event_date_for_sort')
            .dropna(subset=columns, how='all')
            .drop_duplicates('ssn', keep=('last' if best_is_latest else 'first'))
            [['record_id', 'ssn'] + columns]
    )
    return result, record_id_to_single_source_record_pairs(result)

best_name, best_name_source_record_pairs = best_data_from_columns(
    ssa_numident,
    columns=['first_name', 'middle_name', 'last_name'],
)

best_date_of_birth, best_date_of_birth_source_record_pairs = best_data_from_columns(
    ssa_numident,
    columns=['date_of_birth'],
)

best_date_of_death, best_date_of_death_source_record_pairs = best_data_from_columns(
    ssa_numident[ssa_numident.event_type == 'death'],
    columns=['event_date'],
)
best_date_of_death = best_date_of_death.rename(columns={'event_date': 'date_of_death'})

census_numident, census_numident_source_record_pairs = merge_preserving_source_records(
    [best_name, best_date_of_birth, best_date_of_death],
    [best_name_source_record_pairs, best_date_of_birth_source_record_pairs, best_date_of_death_source_record_pairs],
    new_record_id_prefix='census_numident',
    on='ssn',
    how='left',
)
census_numident

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,date_of_death
0,census_numident_0,565-46-1886,Salbador,Robert,NaN,19780317,NaN
1,census_numident_1,221-20-4546,Rebekah,Kandace,Woo,19910627,NaN
2,census_numident_2,116-26-7703,Daniel,Michael,Hernandez,19870904,NaN
3,census_numident_3,594-13-8945,Kelly,Ashley,Safarian,19911110,NaN
4,census_numident_4,702-03-1779,Dalton,Jordan,Ybarra Canalfs,19980818,NaN
...,...,...,...,...,...,...,...
18638,census_numident_18638,381-54-9050,Luke,Devin,Blackman,20230502,NaN
18639,census_numident_18639,154-06-3393,Gary,Ronald,Palumbo,19520217,22240611
18640,census_numident_18640,542-88-1464,John,Victor,Skeeter,19611229,22260313
18641,census_numident_18641,671-15-8285,Leighton,Sophia,Davis,20280207,NaN


### Alternate Name Numident

Wagner and Layne, p. 9:

>  All variants of name information for each SSN are retained in the Alternate Name
Numident file...

In [29]:
alternate_name_numident, alternate_name_numident_source_record_pairs = dedupe_preserving_source_records(
    ssa_numident,
    ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_name', 'last_name'],
    new_record_id_prefix='alternate_name_numident',
)
alternate_name_numident

,record_id,ssn,first_name,middle_name,last_name
0,alternate_name_numident_0,786-77-6454,Evelyn,Nancy,Hinww
1,alternate_name_numident_1,688-88-6377,George,Robert,Dickens
2,alternate_name_numident_2,651-33-9561,Beatrise,Jennie,Fackler
3,alternate_name_numident_3,665-25-7858,Eura,Nadine,Crusen
4,alternate_name_numident_4,875-10-2359,Roberta,Ruth,Muilenburg
...,...,...,...,...,...
19062,alternate_name_numident_19866,203-10-6855,Birdie,Lizzy,Mercer
19063,alternate_name_numident_19867,505-88-8814,Levi,Holden,Small
19064,alternate_name_numident_19868,417-18-3811,Alan,Messiah,Bazan
19065,alternate_name_numident_19869,450-74-3175,Emmi,Everlynn,Will


In [30]:
alternate_name_numident.groupby('ssn').size().describe()

count    18643.000000
mean         1.022743
std          0.149087
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
dtype: float64

In [31]:
alternate_name_numident[alternate_name_numident.ssn.duplicated(keep=False)].sort_values('ssn')

,record_id,ssn,first_name,middle_name,last_name
1187,alternate_name_numident_1187,001-30-0491,Jacqueline,Mary,Gomez
18927,alternate_name_numident_19683,001-30-0491,Jacqueline,NaN,Gomez
16349,alternate_name_numident_16452,003-21-2342,Richard,Richard,Bergeron
849,alternate_name_numident_849,003-21-2342,Richard,Richie,Bergeron
17358,alternate_name_numident_17672,003-84-2583,Diane,Jeanne,Coleman
...,...,...,...,...,...
17545,alternate_name_numident_17910,898-96-4862,Shirlfy,Barbara,Burgos-Valdez
17415,alternate_name_numident_17752,899-07-0109,Jeffrey,Gleng,Hawkins
2971,alternate_name_numident_2971,899-07-0109,Jeffrey,Glenn,Hawkins
2140,alternate_name_numident_2140,899-24-9400,Fred,David,Laduke


### Alternate DOB Numident

Wagner and Layne, p. 9:

> ... while all variants of date of birth data are retained in the Alternate DOB
Numident.

In [32]:
alternate_dob_numident, alternate_dob_numident_source_record_pairs = dedupe_preserving_source_records(
    ssa_numident,
    ssa_numident_source_record_pairs,
    columns_to_dedupe=['ssn', 'date_of_birth'],
    new_record_id_prefix='alternate_dob_numident',
)
alternate_dob_numident

,record_id,ssn,date_of_birth
0,alternate_dob_numident_0,786-77-6454,19191204
1,alternate_dob_numident_1,688-88-6377,19210616
2,alternate_dob_numident_2,651-33-9561,19220113
3,alternate_dob_numident_3,665-25-7858,19220305
4,alternate_dob_numident_4,875-10-2359,19220306
...,...,...,...
18882,alternate_dob_numident_19866,203-10-6855,20291223
18883,alternate_dob_numident_19867,505-88-8814,20291224
18884,alternate_dob_numident_19868,417-18-3811,20291227
18885,alternate_dob_numident_19869,450-74-3175,20291228


In [33]:
alternate_dob_numident.groupby('ssn').size().describe()

count    18643.000000
mean         1.013088
std          0.113655
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          2.000000
dtype: float64

In [34]:
alternate_dob_numident[alternate_dob_numident.ssn.duplicated(keep=False)].sort_values('ssn')

,record_id,ssn,date_of_birth
18802,alternate_dob_numident_19751,005-58-4870,19561009
4485,alternate_dob_numident_4485,005-58-4870,19661009
16993,alternate_dob_numident_17314,013-16-8392,12680418
4870,alternate_dob_numident_4870,013-16-8392,19680818
1307,alternate_dob_numident_1307,013-17-9455,19470921
...,...,...,...
68,alternate_dob_numident_68,892-75-9447,19270319
2834,alternate_dob_numident_2834,893-51-7195,19580101
15799,alternate_dob_numident_15803,893-51-7195,09580101
17933,alternate_dob_numident_18553,897-75-4542,13850509


### Name/DOB Reference File

Wagner and Layne, p. 9:

> The Name and DOB Reference files are reformatted versions of the Census Numident
and includes **all possible combinations of alternate names and dates of birth, as well as
ITIN data**. All of the reference files contain SSN/ITIN and the corresponding PIK. When
an input record is linked to a reference file, the corresponding PIK is assigned. Table 1
presents the number of observations in each of the reference files.

A slightly confusing point: sometimes the Name and DOB reference files are described
as one and the same thing, and sometimes as separate.
I believe this is because **they differ only in how they are "cut" for the PVS process:**
the name reference file is cut by first and last initial,
while the DOB reference file is cut by month and day of birth.

This is described in Wagner and Layne, p.15:

> The [DOBSearch] module matches against a re-split
version of the Numident Name Reference file, splitting the data based on month and day
of birth.

Since we handle the logic of "cutting" in the linkage process itself, we generate
a single reference file here.

Note that unlike for addresses, and unlike for the pre-processing of PVS *input* files
(as opposed to reference files), there is no explicit nickname processing/correction here.
I am fairly sure that is accurate to the real PVS, which I believe assumes that nicknames
would not be present in SSA/tax records (or at least, that the real name would appear
at least once in these records).

In [35]:
name_dob_numident_records, name_dob_numident_records_source_record_pairs = merge_preserving_source_records(
    [alternate_name_numident, alternate_dob_numident],
    [alternate_name_numident_source_record_pairs, alternate_dob_numident_source_record_pairs],
    on='ssn',
    how='left',
    new_record_id_prefix='name_dob_numident_records',
)
name_dob_numident_records

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
0,name_dob_numident_records_0,786-77-6454,Evelyn,Nancy,Hinww,19191204
1,name_dob_numident_records_1,688-88-6377,George,Robert,Dickens,19210616
2,name_dob_numident_records_2,651-33-9561,Beatrise,Jennie,Fackler,19220113
3,name_dob_numident_records_3,665-25-7858,Eura,Nadine,Crusen,19220305
4,name_dob_numident_records_4,875-10-2359,Roberta,Ruth,Muilenburg,19220306
...,...,...,...,...,...,...
19382,name_dob_numident_records_19382,203-10-6855,Birdie,Lizzy,Mercer,20291223
19383,name_dob_numident_records_19383,505-88-8814,Levi,Holden,Small,20291224
19384,name_dob_numident_records_19384,417-18-3811,Alan,Messiah,Bazan,20291227
19385,name_dob_numident_records_19385,450-74-3175,Emmi,Everlynn,Will,20291228


In [36]:
name_dob_numident_records[name_dob_numident_records.ssn.duplicated(keep=False)].sort_values('ssn')

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
1282,name_dob_numident_records_1282,001-30-0491,Jacqueline,Mary,Gomez,19460824
19242,name_dob_numident_records_19242,001-30-0491,Jacqueline,NaN,Gomez,19460824
16603,name_dob_numident_records_16603,003-21-2342,Richard,Richard,Bergeron,19430622
936,name_dob_numident_records_936,003-21-2342,Richard,Richie,Bergeron,19430622
3579,name_dob_numident_records_3579,003-84-2583,Diane,Jeamne,Coleman,19610314
...,...,...,...,...,...,...
17819,name_dob_numident_records_17819,898-96-4862,Shirlfy,Barbara,Burgos-Valdez,19240302
17688,name_dob_numident_records_17688,899-07-0109,Jeffrey,Gleng,Hawkins,19580911
3146,name_dob_numident_records_3146,899-07-0109,Jeffrey,Glenn,Hawkins,19580911
2284,name_dob_numident_records_2284,899-24-9400,Fred,David,Laduke,19530914


#### Incorporating people with ITINs

Individual Taxpayer Identification Numbers (ITINs) can be issued to people who are required to file
federal taxes but are not eligible for a Social Security Number.
The most common reason for this is being an undocumented immigrant and therefore not being authorized
to work in the United States.

People without SSNs used to be impossible to assign PIKs to.
In 2011 the NORC report stated (p. 38, footnote 19):

> NORC understands that the Census Bureau has undertaken an effort to enhance the PVS reference files with IRS
files that include Individual Taxpayer Identification Numbers (ITIN). For those people who are required to file a tax
return but do not have, and may not want an SSN—such as a non-U.S. citizen—the IRS issues the taxpayer an ITIN.
This enhancement to the PVS reference file may help to match more non-U.S citizens.

By 2014 (Wagner and Layne, p. 5):

> One of the key enhancements [made in recent years] increased the coverage of the reference files by
including records for persons with Individual Taxpayer Identification Numbers assigned
by the Internal Revenue Service (ITINs) to [along with?] the SSN-based Numident data. 

I have not found a specific description of how ITIN records are constructed in any of the
publicly-available sources.
This may be because it is straightforward, or because the tax data schema is confidential.
I assume that only IRS data is used, since no other data source that I am aware of would
report ITIN.

It is stated that the ITIN records are created directly from tax filings and not
from ITIN applications (Brown et al. p. 29, footnote 16), which is convenient
because the tax filing data is what we can simulate with
pseudopeople:

> The NUMIDENT provides the PII on the SSN-holder from the issuing agency (SSA), and that PII is used in SSN
verification. **For ITINs, the Census Bureau does not have access to the ITIN applications** to the issuing agency (IRS),
so name and DOB verification of ITINs is less reliable.

"Less reliable" is a bit confusing here, because as stated above when generating
the simulated tax data, IRS data should not contain date of birth at all.
Here, we have stayed true to this by omitting it entirely.

We assume that only 1040 filings would be used for this purpose; we wouldn't expect ITINs to
show up on employer-filed W-2/1099 forms.

In [37]:
# Analogous to the process of getting alternate names and dates of birth
# from SSA, we retain all versions of the name from taxes.
taxes_1040_with_itins = taxes_1040[taxes_1040.ssn.notnull() & taxes_1040.ssn.str.startswith('9')]
taxes_1040_with_itins_source_record_pairs = taxes_1040_source_record_pairs[taxes_1040_source_record_pairs.record_id.isin(taxes_1040_with_itins.record_id)]
name_for_itins, name_for_itins_source_record_pairs = dedupe_preserving_source_records(
    taxes_1040_with_itins,
    taxes_1040_with_itins_source_record_pairs,
    columns_to_dedupe=['ssn', 'first_name', 'middle_initial', 'last_name'],
    new_record_id_prefix='name_for_itins',
)
name_for_itins = name_for_itins.rename(columns={'middle_initial': 'middle_name'})
name_for_itins

,record_id,ssn,first_name,middle_name,last_name
0,name_for_itins_31,974-55-3537,Ryan,T,Osuna
1,name_for_itins_49,950-88-5166,Tammy,P,White
2,name_for_itins_52,923-74-4942,Emili,S,Grewal
3,name_for_itins_55,919-88-1971,Carol,B,Cox
4,name_for_itins_76,945-76-4582,Brandon,A,Amezcua-Bahena
...,...,...,...,...,...
622,name_for_itins_38535,943-53-1588,Keith,J,Velazquez
623,name_for_itins_38584,926-80-1255,Carl,S,Saulsby
624,name_for_itins_38743,910-79-0552,Kayla,J,Morley
625,name_for_itins_38961,999-83-8127,Keith,C,Esquivel


In [38]:
name_for_itins.groupby('ssn').size().describe()

count    509.000000
mean       1.231827
std        0.462457
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        4.000000
dtype: float64

In [39]:
# Make sure these are disjoint sets of SSN values -- this could only happen
# if an SSN value in the Numident were corrupted into the ITIN range
assert set(name_dob_numident_records.ssn) & set(name_for_itins.ssn) == set()

In [40]:
name_dob_reference_file, name_dob_reference_file_source_record_pairs = concat_preserving_source_records(
    [name_dob_numident_records, name_for_itins],
    [name_dob_numident_records_source_record_pairs, name_for_itins_source_record_pairs],
    new_record_id_prefix='name_dob_reference_file',
)

name_dob_reference_file

,record_id,ssn,first_name,middle_name,last_name,date_of_birth
0,name_dob_reference_file_0,786-77-6454,Evelyn,Nancy,Hinww,19191204
1,name_dob_reference_file_1,688-88-6377,George,Robert,Dickens,19210616
2,name_dob_reference_file_2,651-33-9561,Beatrise,Jennie,Fackler,19220113
3,name_dob_reference_file_3,665-25-7858,Eura,Nadine,Crusen,19220305
4,name_dob_reference_file_4,875-10-2359,Roberta,Ruth,Muilenburg,19220306
...,...,...,...,...,...,...
20009,name_dob_reference_file_20009,943-53-1588,Keith,J,Velazquez,NaN
20010,name_dob_reference_file_20010,926-80-1255,Carl,S,Saulsby,NaN
20011,name_dob_reference_file_20011,910-79-0552,Kayla,J,Morley,NaN
20012,name_dob_reference_file_20012,999-83-8127,Keith,C,Esquivel,NaN


### GeoBase Reference File

Wagner and Layne, p. 9:

> PVS creates three other sets of reference
files containing Numident data: the **GeoBase Reference File**, the Name Reference File,
and the DOB Reference file.
The GeoBase Reference File appends addresses from administrative records attached
to Numident data, including all possible combinations of alternate names and dates of
birth for SSN. Addresses from administrative records are edited and processed through
commercial software product to clean and standardize address data. ITIN data is also
incorporated into the Geobase.

Luque and Wagner, p. 5:

> Reference files contain data from the Numident file enhanced with address
data obtained from federal AR [administrative records] files.<sup>8</sup>
The reference files, thus, contain all variants of a person’s name, DOB,
and sex, as well as current and recent addresses. These reference files are
referred to as the (PVS) Geobase reference file since addresses (a geographic component)
are appended to each person record.<sup>9</sup> It is important to note that there are
multiple Geobase reference files that are created depending on the vintage of the
incoming file to be processed through PVS.

> <sup>8</sup> Namely, data from the IRS, Department of Housing and Urban Development,
several files from the Department of Health and Human Services, and Selective Service.

> <sup>9</sup> In particular, the address data is cleaned and standardized and used
to construct a variable called GEOKEY. The GEOKEY variable is constructed as a subset
of the full address, and then is appended to the Numident data to create the
PVS Geobase Reference file.

We only have IRS data to use for addresses, and specifically only W-2/1099 data,
which is a limitation of this case study.
I can't find a concrete definition of "recent" -- as noted above, we use 5 years
of IRS data.
This may be longer than the true window, but this may end up making up for
the lack of non-IRS data sources, and get us closer to a realistic number of
alternate addresses.

Also, our address data comes out of pseudopeople already parsed into address parts
like street name, etc.
For more realism, pseudopeople should output a single string that we have to (imperfectly) parse apart.

I haven't been able to find out more about what kind of "subset" the geokey is.
It is unclear to me why geokey is "interesting" since it is just derived from the
address parts.

In [41]:
address_cols = list(taxes.filter(like='mailing_address').columns)

def standardize_address_part(column):
    return (
        column
            # Remove leading or trailing whitespace
            .str.strip()
            # Turn any strings of consecutive whitespace into a single space
            .str.split().str.join(' ')
            # Normalize case
            .str.upper()
            # Normalize the word street as described in the example quoted above
            # In reality, there would be many rules like this
            .str.replace('\b(STREET|STR)\b', 'ST', regex=True)
            # Make sure missingness is represented consistently
            .replace('', np.nan)
    )

tax_addresses = (
    taxes.set_index(['record_id', 'ssn'])
        [address_cols]
        .apply(standardize_address_part)
        .reset_index()
)
addresses_by_ssn, addresses_by_ssn_source_record_pairs = dedupe_preserving_source_records(
    tax_addresses,
    taxes_source_record_pairs,
    columns_to_dedupe=['ssn'] + address_cols,
    new_record_id_prefix='addresses_by_ssn',
)
addresses_by_ssn

,record_id,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,addresses_by_ssn_0,794-23-1522,NaN,NaN,NaN,14011,ANYTOWN,OH,00000
1,addresses_by_ssn_1,004-53-9627,1934,CYPRESS AVE,NaN,NaN,ANYTOWN,WA,00000
2,addresses_by_ssn_2,273-52-7750,57,SHADOWOOD DR,NaN,NaN,ANYTOWN,WA,00000
3,addresses_by_ssn_3,384-45-8523,10881,MAPLE AV,NaN,NaN,ANYTOWN,WA,00000
4,addresses_by_ssn_4,858-84-4517,103,71ST ST W,NaN,NaN,ANYTOWN,WA,00000
...,...,...,...,...,...,...,...,...,...
32567,addresses_by_ssn_92725,581-70-8056,1105,LEMONWOOD CT,NaN,NaN,ANYTOWN,WA,00000
32568,addresses_by_ssn_92727,081-70-8056,1105,LEMONWOOD CT,NaN,NaN,ANYTOWN,WA,00000
32569,addresses_by_ssn_92735,518-45-2600,716,WAKEMA RD,NaN,NaN,ANYTOWN,WA,00000
32570,addresses_by_ssn_92739,079-75-0734,9358,E 58TH ST,NaN,NaN,ANYTOWN,WA,00000


In [42]:
num_addresses = addresses_by_ssn.groupby('ssn').size().sort_values()
num_addresses

ssn
000-79-9923    1
545-28-0443    1
545-31-3602    1
545-35-4995    1
545-80-0380    1
              ..
543-85-7977    7
682-87-8833    7
347-72-4591    7
056-61-5935    7
144-18-4623    9
Length: 19291, dtype: int64

In [43]:
# Show some SSNs with a lot of address variation
addresses_by_ssn[addresses_by_ssn.ssn.isin(num_addresses.tail(10).index)].sort_values('ssn')

,record_id,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
30061,addresses_by_ssn_82256,007-67-7210,34,NaN,NaN,NaN,ANYTOWN,WA,00000
22753,addresses_by_ssn_49505,007-67-7210,112,WSTTERN AV,NaN,NaN,ANYTOWN,WA,00000
3518,addresses_by_ssn_3524,007-67-7210,34,BOWEN CIR SW,NaN,NaN,ANYTOWN,WA,00000
30060,addresses_by_ssn_82254,007-67-7210,24,BOWEN CIR SW,NaN,NaN,ANYTOWN,WA,00000
15997,addresses_by_ssn_26814,007-67-7210,34,BOWEN CIR SW,NaN,NaN,ANYTOWN,WA,NaN
...,...,...,...,...,...,...,...,...,...
21117,addresses_by_ssn_43903,859-49-8970,1608,W NORT AVENUE,NaN,NaN,ANYTOWN,NaN,00000
30585,addresses_by_ssn_84462,859-49-8970,69,S HWY 701,NaN,NaN,ANYTOWH,WA,NaN
21116,addresses_by_ssn_43902,859-49-8970,1608,W NORT AVENUE,NaN,NaN,ANYTOWN,WA,NaN
17091,addresses_by_ssn_29978,859-49-8970,69,S HWY 701,NaN,NaN,ANYTOWN,WA,00000


In [44]:
# Rough estimate of how many rows we should have in our reference file, once we do this Cartesian product
(
    len(name_dob_reference_file) *
    addresses_by_ssn.groupby('ssn').size().mean()
)

33018.79446373956

In [45]:
geobase_reference_file, geobase_reference_file_source_record_pairs = merge_preserving_source_records(
    [name_dob_reference_file, addresses_by_ssn],
    [name_dob_reference_file_source_record_pairs, addresses_by_ssn_source_record_pairs],
    on='ssn',
    how='left',
    new_record_id_prefix='geobase_reference_file',
)
geobase_reference_file

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,mailing_address_city,mailing_address_state,mailing_address_zipcode
0,geobase_reference_file_0,786-77-6454,Evelyn,Nancy,Hinww,19191204,34,BOWEN CIR SW,NaN,NaN,ANYTOWN,WA,00000
1,geobase_reference_file_1,688-88-6377,George,Robert,Dickens,19210616,20340,W ORANGE GROVE AVE,NaN,NaN,ANYTOWN,WA,00000
2,geobase_reference_file_2,651-33-9561,Beatrise,Jennie,Fackler,19220113,9739,73RD AVN,NaN,NaN,ANYTOWN,WA,00000
3,geobase_reference_file_3,665-25-7858,Eura,Nadine,Crusen,19220305,2,BEEHLER AVE,NaN,NaN,ANYTOWN,WA,00000
4,geobase_reference_file_4,665-25-7858,Eura,Nadine,Crusen,19220305,2,BEEHLER AVE,NaN,NaN,ANYTON,WA,00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32182,geobase_reference_file_32182,943-53-1588,Keith,J,Velazquez,NaN,120,MONTGALL AVE,NaN,NaN,ANYTOWN,WA,00000
32183,geobase_reference_file_32183,926-80-1255,Carl,S,Saulsby,NaN,210,W 66TH ST,NaN,NaN,ANYTOWN,WA,00000
32184,geobase_reference_file_32184,910-79-0552,Kayla,J,Morley,NaN,4692,CISCO COURT,NaN,NaN,ANYTOWN,WA,00000
32185,geobase_reference_file_32185,999-83-8127,Keith,C,Esquivel,NaN,38,FRANCES ST,NaN,NaN,ANYTOWN,WA,00000


## Track ground truth for reference files

In [46]:
# TODO: This is really slow compared to pandas' mode.
# Mode should be added into Modin as a first-class citizen.
def mode(s):
    val_counts = s.value_counts()
    return val_counts.loc[val_counts == val_counts.max()].reset_index().loc[:, val_counts.index.name].rename(None)

def get_simulants_for_record_ids(record_ids, ground_truth=source_record_ground_truth):
    return tuple(ground_truth.loc[record_id] for record_id in record_ids)

def get_simulants_of_source_records(source_record_pairs, filter_record_ids=None, ground_truth=source_record_ground_truth):
    if filter_record_ids is not None:
        source_record_pairs = source_record_pairs.pipe(filter_record_ids)
    grouped_simulant_id = lambda: (
        source_record_pairs
            .merge(ground_truth.reset_index().rename(columns={'record_id': 'source_record_id'}), on='source_record_id')
            .groupby('record_id')
            .simulant_id
    )
    
    return (
        # NOTE: You'd expect to be able to just call .agg([mode, 'nunique']) instead of joining.
        # This is probably slower than that would be, but that doesn't work due to a Modin bug:
        # https://github.com/modin-project/modin/issues/6600
        pd.DataFrame(grouped_simulant_id().agg(mode)).merge(pd.DataFrame(grouped_simulant_id().agg('nunique').rename('nunique')), left_index=True, right_index=True)
    )

### Census Numident

In [47]:
census_numident_ground_truth = get_simulants_of_source_records(census_numident_source_record_pairs)
census_numident_ground_truth

,simulant_id,nunique
record_id,,
census_numident_0,0_15796,1
census_numident_1,0_14159,1
census_numident_10,0_808,1
census_numident_100,0_17531,1
census_numident_1000,0_14447,1
...,...,...
census_numident_9995,0_2156,1
census_numident_9996,0_582,1
census_numident_9997,0_6239,1


In [48]:
census_numident_ground_truth['nunique'].describe()

count    18643.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: nunique, dtype: float64

### Alternate Name Numident

In [49]:
# We take the most common ground truth value.
alternate_name_numident_ground_truth = get_simulants_of_source_records(alternate_name_numident_source_record_pairs)
alternate_name_numident_ground_truth

,simulant_id,nunique
record_id,,
alternate_name_numident_0,0_19979,1
alternate_name_numident_1,0_6846,1
alternate_name_numident_10,0_5125,1
alternate_name_numident_100,0_14376,1
alternate_name_numident_1000,0_18224,1
...,...,...
alternate_name_numident_9995,0_6602,1
alternate_name_numident_9996,0_15024,1
alternate_name_numident_9997,0_19784,1


In [50]:
# Again, as shown above, there are no SSN collisions.
alternate_name_numident_ground_truth['nunique'].describe()

count    19067.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: nunique, dtype: float64

### Alternate DOB Numident

In [51]:
alternate_dob_numident_ground_truth = get_simulants_of_source_records(alternate_dob_numident_source_record_pairs)
alternate_dob_numident_ground_truth

,simulant_id,nunique
record_id,,
alternate_dob_numident_0,0_19979,1
alternate_dob_numident_1,0_6846,1
alternate_dob_numident_10,0_5125,1
alternate_dob_numident_100,0_14376,1
alternate_dob_numident_1000,0_18224,1
...,...,...
alternate_dob_numident_9995,0_6602,1
alternate_dob_numident_9996,0_15024,1
alternate_dob_numident_9997,0_19784,1


In [52]:
alternate_dob_numident_ground_truth['nunique'].describe()

count    18887.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: nunique, dtype: float64

### Name/DOB Reference File

In [53]:
name_dob_reference_file_ground_truth = get_simulants_of_source_records(name_dob_reference_file_source_record_pairs)
name_dob_reference_file_ground_truth

,simulant_id,nunique
record_id,,
name_dob_reference_file_0,0_19979,1
name_dob_reference_file_1,0_6846,1
name_dob_reference_file_10,0_19806,1
name_dob_reference_file_100,0_17351,1
name_dob_reference_file_1000,0_8162,1
...,...,...
name_dob_reference_file_9995,0_5004,1
name_dob_reference_file_9996,0_2346,1
name_dob_reference_file_9997,0_2669,1


In [54]:
name_dob_reference_file_ground_truth['nunique'].describe()

count    20014.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: nunique, dtype: float64

### GeoBase Reference File

In [55]:
geobase_reference_file_ground_truth_naive = get_simulants_of_source_records(geobase_reference_file_source_record_pairs)
geobase_reference_file_ground_truth_naive

,simulant_id,nunique
record_id,,
geobase_reference_file_0,0_19979,1
geobase_reference_file_1,0_6846,1
geobase_reference_file_10,0_4701,1
geobase_reference_file_100,0_1109,1
geobase_reference_file_1000,0_6757,1
...,...,...
geobase_reference_file_9995,0_8785,2
geobase_reference_file_9996,0_8785,1
geobase_reference_file_9997,0_4990,1


In [56]:
# Now there are some collisions, due to "borrowed SSN" noise
geobase_reference_file_ground_truth_naive['nunique'].describe()

count    32187.000000
mean         1.039892
std          0.201651
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          3.000000
Name: nunique, dtype: float64

In [57]:
# The most collisions on one SSN
most_collisions_record_id = geobase_reference_file_ground_truth_naive.reset_index().sort_values('nunique', ascending=False).iloc[0].record_id
most_collisions_source_record_ids = geobase_reference_file_source_record_pairs[geobase_reference_file_source_record_pairs.record_id == most_collisions_record_id].source_record_id
most_collisions_tax_filings = taxes[taxes.merge(taxes_source_record_pairs, on='record_id').source_record_id.isin(most_collisions_source_record_ids)]
most_collisions_tax_filings

,record_id,household_id,first_name,middle_initial,last_name,ssn,mailing_address_street_number,mailing_address_street_name,mailing_address_unit_number,mailing_address_po_box,...,tax_form,employer_id,wages,employer_name,employer_street_number,employer_street_name,employer_unit_number,employer_city,employer_state,employer_zipcode
3379,taxes_3379,0_65,Elizabeth,O,Amezcua-Bahena,442-50-6000,412,brentwood r norteast,NaN,NaN,...,1040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18722,taxes_18722,0_65,Elisabeth,O,Amez,442-50-6000,412,brentwood r norteast,NaN,NaN,...,1040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61167,taxes_61167,0_65,Brandon,A,Amez,442-50-6000,412,brentwood r norteast,NaN,NaN,...,W2,95,57030,Pikes Creek Campground,6300,east lincoln ave,NaN,Anytown,WA,00000


In [58]:
# Correct value: who actually has the SSN
ssa_numident[ssa_numident.ssn.isin(most_collisions_tax_filings.ssn.unique())].merge(ssa_numident_ground_truth, on='record_id')

,record_id,ssn,first_name,middle_name,last_name,date_of_birth,sex,event_type,event_date,simulant_id
0,ssa_numident_14694,442-50-6000,Elizabeth,Olivia,Amezcua-Bahena,20130907,Female,creation,20130907,0_148


In [59]:
# Let's prioritize the use of SSA records
geobase_reference_file_ground_truth_ssa_only = get_simulants_of_source_records(
    geobase_reference_file_source_record_pairs,
    filter_record_ids=lambda df: df[df.source_record_id.str.startswith('ssa_numident_')],
)
geobase_reference_file_ground_truth_ssa_only

,simulant_id,nunique
record_id,,
geobase_reference_file_0,0_19979,1
geobase_reference_file_1,0_6846,1
geobase_reference_file_10,0_4701,1
geobase_reference_file_100,0_1109,1
geobase_reference_file_1000,0_6757,1
...,...,...
geobase_reference_file_9995,0_8785,1
geobase_reference_file_9996,0_8785,1
geobase_reference_file_9997,0_4990,1


In [60]:
geobase_reference_file_ground_truth_ssa_only['nunique'].describe()

count    31331.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: nunique, dtype: float64

In [61]:
geobase_reference_file_ground_truth = (
    geobase_reference_file_ground_truth_naive.join(geobase_reference_file_ground_truth_ssa_only, lsuffix='_naive', rsuffix='_ssa_only', how='outer')
        .assign(
            # When there are no SSA records (ITIN-based), use the standard mode
            simulant_id=lambda df: df.simulant_id_ssa_only.fillna(df.simulant_id_naive),
            nunique=lambda df: df.nunique_naive,
        )
        [['simulant_id', 'nunique']]
)
geobase_reference_file_ground_truth

,simulant_id,nunique
record_id,,
geobase_reference_file_0,0_19979,1
geobase_reference_file_1,0_6846,1
geobase_reference_file_10,0_4701,1
geobase_reference_file_100,0_1109,1
geobase_reference_file_1000,0_6757,1
...,...,...
geobase_reference_file_9995,0_8785,2
geobase_reference_file_9996,0_8785,1
geobase_reference_file_9997,0_4990,1


In [62]:
assert set(geobase_reference_file.record_id) == set(geobase_reference_file_ground_truth.index)

### Get ground truth by SSN

In [63]:
all_ssn_simulant_pairs = pd.concat([
    census_numident.set_index("record_id")[["ssn"]].join(pd.DataFrame(census_numident_ground_truth)),
    alternate_name_numident.set_index("record_id")[["ssn"]].join(pd.DataFrame(alternate_name_numident_ground_truth)),
    alternate_dob_numident.set_index("record_id")[["ssn"]].join(pd.DataFrame(alternate_dob_numident_ground_truth)),
    name_dob_reference_file.set_index("record_id")[["ssn"]].join(pd.DataFrame(name_dob_reference_file_ground_truth)),
    geobase_reference_file.set_index("record_id")[["ssn"]].join(pd.DataFrame(geobase_reference_file_ground_truth)),
])
all_ssn_simulant_pairs

,ssn,simulant_id,nunique
record_id,,,
census_numident_0,565-46-1886,0_15796,1
census_numident_1,221-20-4546,0_14159,1
census_numident_2,116-26-7703,0_15183,1
census_numident_3,594-13-8945,0_7921,1
census_numident_4,702-03-1779,0_3197,1
...,...,...,...
geobase_reference_file_32182,943-53-1588,0_7677,1
geobase_reference_file_32183,926-80-1255,0_1328,1
geobase_reference_file_32184,910-79-0552,0_23720,1


In [64]:
# The reference file records with a given SSN all have the same (primary) simulant ID
# contributing to them
assert (all_ssn_simulant_pairs.groupby('ssn').simulant_id.nunique() == 1).all()

In [65]:
ssn_to_simulant = all_ssn_simulant_pairs.groupby('ssn').simulant_id.first()
ssn_to_simulant

ssn
001-02-4588    0_11023
001-11-6330     0_3108
001-16-0077     0_3922
001-17-9511    0_17690
001-25-8258    0_22503
                ...   
999-92-3977    0_10234
999-99-9858     0_7327
9s6-77-5983     0_8607
9zs-So-7561    0_13082
9|s-51-3204    0_17739
Name: simulant_id, Length: 19152, dtype: object

## Save results

In [66]:
files = {
    'census_2030': (census_2030, census_2030_ground_truth),
    'census_numident': (census_numident, census_numident_ground_truth),
    'alternate_name_numident': (alternate_name_numident, alternate_name_numident_ground_truth),
    'alternate_dob_numident': (alternate_dob_numident, alternate_dob_numident_ground_truth),
    'geobase_reference_file': (geobase_reference_file, geobase_reference_file_ground_truth),
    'name_dob_reference_file': (name_dob_reference_file, name_dob_reference_file_ground_truth),
}

In [67]:
reference_files = [census_numident, alternate_name_numident, alternate_dob_numident, geobase_reference_file, name_dob_reference_file]
# TODO: Rename the ssn column to explicitly include itins, since this is confusing
all_ssns_itins_in_reference_files = pd.concat([df[["ssn"]] for df in reference_files], ignore_index=True)
ssn_to_pik = (
    all_ssns_itins_in_reference_files.drop_duplicates()
        .reset_index().rename(columns={'index': 'pik'})
        .set_index('ssn').pik
)
ssn_to_pik

ssn
565-46-1886        0
221-20-4546        1
116-26-7703        2
594-13-8945        3
702-03-1779        4
               ...  
943-53-1588    88779
926-80-1255    88780
910-79-0552    88781
999-83-8127    88782
997-51-7349    88783
Name: pik, Length: 19152, dtype: int64

In [68]:
pik_to_simulant = (
    ssn_to_simulant.reset_index()
        .assign(pik=lambda df: df.ssn.map(ssn_to_pik))
        .set_index("pik")
        .simulant_id
)
pik_to_simulant

pik
12710    0_11023
6037      0_3108
4961      0_3922
4367     0_17690
17377    0_22503
          ...   
88752    0_10234
88402     0_7327
88649     0_8607
88631    0_13082
88351    0_17739
Name: simulant_id, Length: 19152, dtype: object

In [69]:
import os
import shutil
from pathlib import Path

def remove_path(path):
    path = Path(path)
    if path.is_file():
        os.remove(path)
    elif path.exists():
        shutil.rmtree(path)

for file_name, (file, ground_truth) in files.items():
    # Add a unique record ID -- could do this within the pipeline, but then it's harder to match up the ground truth
    assert file.record_id.is_unique and ground_truth.index.is_unique
    assert set(file.record_id) == set(ground_truth.index)

    if file_name != 'census_2030':
        file['pik'] = file.ssn.map(ssn_to_pik)
        assert file.pik.notnull().all()

    ground_truth = ground_truth.reset_index()

    # HACK: Why does this end up having lots of ints?
    # This must be an issue in pseudopeople!
    if 'age' in file.columns:
        file['age'] = file.age.astype(str)

    file_path = Path(f'{output_dir}/{file_name}_{data_to_use}.parquet')
    remove_path(file_path)
    file.to_parquet(file_path)

    ground_truth_path = Path(f'{output_dir}/{file_name}_ground_truth_{data_to_use}.parquet')
    remove_path(ground_truth_path)
    ground_truth.to_parquet(ground_truth_path)

In [70]:
pik_to_simulant_path = Path(f'{output_dir}/pik_to_simulant_ground_truth_{data_to_use}.parquet')
remove_path(pik_to_simulant_path)
pik_to_simulant.reset_index().to_parquet(pik_to_simulant_path)

In [71]:
# Convert this notebook to a Python script
! cd .. && ./convert_notebook.sh generate_simulated_data/generate_simulated_data_small_sample

[NbConvertApp] Converting notebook generate_simulated_data/generate_simulated_data_small_sample.ipynb to python


In [72]:
! date

Wed Sep 27 11:33:30 AM PDT 2023
